# **Tutorial 2: Prediksi Band Gap Eksperimental - Random Forest**

Sekarang coba kita prediksi band gap eksperimental dari data yang sama dengan algoritma Random Forest.

Seperti biasa, import dulu package yang diperlukan. Alih-alih kita import nanti, kita langsung import di awal.

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import plotly.express as px
import plotly.graph_objects as go 

Kita panggil datasetnya, pisah data feature dan target, bagi jadi data training dan test.

In [3]:
# Import dataset
filepath = "../datasets/expt_gap.csv"
data = pd.read_csv(filepath)

# Pisahkan fitur dan target
X = data.drop(columns=['composition','gap expt'])
y = data['gap expt']

# Bagi data jadi data training dan testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

Sekarang kita bangun model Random Forest. Kali ini, model Random Forest kita panggil lewat Pipeline lagi. Untuk kasus kita ini, nggak wajib sih pakai Pipeline, cuma it's a good habit buat pakai Pipeline.

Perhatikan kalau kita juga definisikan kfold. Lagi, kita pakai 5-fold cross-validation.

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate

scoring = ["r2", "neg_mean_squared_error"]

rf_pipeline = Pipeline([
    ("model", RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    ))
])

kfold = KFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

Oke. Jalankan cross-validation.

In [5]:
rf_hasil_cv = cross_validate(
    rf_pipeline,
    X_train, y_train,
    cv=kfold,
    scoring=scoring,          
    return_train_score=True,
    n_jobs=-1
)

Performa model selama cross-validation:

In [6]:
rf_r2 = rf_hasil_cv["test_r2"]
rf_mse = -rf_hasil_cv["test_neg_mean_squared_error"]

print("RF CV R² per fold:", rf_r2)
print("RF CV Mean R²:", rf_r2.mean())
print("RF CV Std R²:", rf_r2.std())

print("RF CV MSE per fold:", rf_mse)
print("RF CV Mean MSE:", rf_mse.mean())

RF CV R² per fold: [0.75424475 0.73906364 0.83117549 0.81000551 0.76827089]
RF CV Mean R²: 0.7805520553146709
RF CV Std R²: 0.03462493033370116
RF CV MSE per fold: [0.62374696 0.46988974 0.31384965 0.42546028 0.5417225 ]
RF CV Mean MSE: 0.4749338269418192


Oke, sekarang training terakhir dengan data training yang komplit dilanjutkan prediksi pakai data test yang belum pernah dilihat oleh model kita.

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

rf_pipeline.fit(X_train, y_train)

y_test_pred_rf = rf_pipeline.predict(X_test)

print("RF Test R²:", r2_score(y_test, y_test_pred_rf))
print("RF Test MSE:", mean_squared_error(y_test, y_test_pred_rf))
print("RF Test MSE:", np.sqrt(mean_squared_error(y_test, y_test_pred_rf)))

RF Test R²: 0.8095928597177398
RF Test MSE: 0.3959583738291768


Nilai R^2-nya membuat semangat 🤩.

Mari kita tambahkan hasil prediksi ke tabel data full kita.

In [8]:
data['predicted_gap'] = rf_pipeline.predict(X)
data['percentage_error'] = np.abs(data['predicted_gap'] - data['gap expt']) / data['gap expt'] * 100

Sekarang, coba kita plot hasil prediksi vs band gap eksperimental yang asli dari data.

In [9]:
reference_line = go.Scatter(
    x=[0, 10],
    y=[0,10] ,
    line=dict(color='red', dash='dash'),
    mode="lines",
    showlegend=False
)

fig = px.scatter(data,
    x="predicted_gap",
    y="gap expt",
    hover_name="composition",
    color="percentage_error",
    title="Predicted vs Experimental Band Gaps",
    color_continuous_scale=px.colors.sequential.Bluered,
    width=1200,
    height=800,
)

fig.add_trace(reference_line)
fig.show()

Wow masya Allah 🍵.

# **Menyimpan Model yang Sudah Ditraining**

Kita bisa simpan model yang sudah ditraining untuk digunakan di masa yang akan datang. Kita bisa pakai joblib untuk ini.

So, import joblib dulu.

In [10]:
import joblib

Di folder ini kita akan simpan model kita. Pastikan dulu kalau foldernya sudah ada. Kalau belum, bisa dibuat dulu otherwise Python bakal ngelempar error ke kita.

In [11]:
output_dir = "../models/"

Oke, kita simpan model kita.

In [ ]:
# Simpan model yang sudah ditraining
model_filename = "rf_bandgap_model.joblib"

joblib.dump(rf_pipeline, output_dir + model_filename)

['../models/rf_bandgap_model.joblib']

Daftar feature yang kita pakai ketika ngedevelop model juga penting, jadi kita simpan juga.

In [ ]:
# List fitur juga penting
fitur = X.columns.tolist()

joblib.dump(fitur, output_dir + "rf_bandgap_features.joblib")

['../models/rf_bandgap_features.joblib']

Jadi, ketika kita sudah merasa puas dengan model yang kita develop, kita bisa simpan untuk digunakan di kemudian hari. Begitu kita punya data baru dan butuh info band gap, kita bisa coba prediksi dengan model yang sudah kita develop ini.

Untuk memanggil model yang sudah ditraining, jalankan

```
import joblib

model = joblib.load("path_ke_model_file.joblib")
```

Misal data baru sudah kita panggil di notebook sebagai `X_new`, untuk memprediksi kita tinggal jalankan

```
y_new = model.predict(X_new)
```